# Irregular timestamps

TimeGPT can handle data with irregular timestamps. Simply specify the `freq` parameter with the right frequency of your data.

In [ ]:
import pandas as pd
from nixtla import NixtlaClient

In [ ]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
load_dotenv()
nixtla_client = NixtlaClient()

In [ ]:
# Read data
# Dates for the weekends are missing
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/openbb/pltr.csv')

# Forecast
# We use B for the freq, as only business days are represented in the dataset
forecast_df = nixtla_client.forecast(
    df=df, 
    h=14, 
    freq='B',
    time_col='date', 
    target_col='Close',
)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


For more details on handling datasets with irregular timesteps, check out our [tutorial](https://docs.nixtla.io/docs/tutorials-irregular_timestamps). 